#**This notebook was ran on google colab**
Mohamed Boujlida 2021

##First we need to install the necessary libraries to detect the *table*
#In this notebook we will use [pdf2image](https://pdf2image.readthedocs.io/en/latest/reference.html) to change the pdf to images and [Layout parser](https://layout-parser.readthedocs.io/en/latest/) to preform OCR

In [ ]:
!pip install layoutparser[ocr]
## we will use a pretrained model to detect the position of the table
!pip3 install 'git+https://github.com/facebookresearch/detectron2.git@v0.4#egg=detectron2'
!pip install pdf2image
!apt-get install poppler-utils 

Reading package lists... Done
Building dependency tree       
Reading state information... Done
poppler-utils is already the newest version (0.62.0-2ubuntu2.12).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.


# Table Detection
###now if you are working on colab you might want to use google drive to save your data, the below is code for mounting drive
## Get our data from google drive (optional)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
##change the directory to the file where I uploaded the neceessary documents
os.chdir("write your desired directory here ")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Import the necessary libraries to detect the table 

In [ ]:
import layoutparser as lp 

import matplotlib.pyplot as plt
%matplotlib inline 

import pandas as pd
import numpy as np
import cv2
import csv

try:
    from PIL import Image
except ImportError:
    import Image

## Read the the KEY  in order to preform OCR 
### layout parser uses Google cloud Vision api in order to prefrom OCR learning more about it here [Google cloud Vision](https://cloud.google.com/vision/docs/ocr) and for the key [Key](https://cloud.google.com/docs/authentication/api-keys)

In [ ]:
ocr_agent = lp.GCVAgent.with_credential("write the path of your token here , it should end with .json",languages = ['choose the desired langauge, if it is english just write "en", if it is japanese , wirte "ja"'])

## get the deep learning model to preform table detection

In [ ]:
model = lp.Detectron2LayoutModel('lp://TableBank/faster_rcnn_R_101_FPN_3x/config',
                                 label_map={0: "Table"})

#change pdf to pictures (optional)



In [ ]:
change_to_image=True
first_page=0
#last page +1
last_page=159
if change_to_image:
  input_path='input path for the pdf'
  output_path="output path for transformed images from the pdf"
  from pdf2image import convert_from_path 
  # Store Pdf with convert_from_path function
  for i in range(first_page,last_page):
    images = convert_from_path(input_path,first_page=i,last_page=i+1,dpi=1000)
    images[0].save(output_path+'page'+ str(i) +'.jpg', 'JPEG')

##set your paths

In [ ]:
#set the output file name 
output_file="output1"
path_output=os.path.join("output path for excel file",output_file)
os.mkdir(path_output)

#now let's start the algorithm
## I based the algorithm on the one  [here](https://gist.github.com/huks0/e48d604fc9dd91731bc687d6e3933db4#file-cellrecognition-py)
### my contriubtion is using layout parser to detect the table first, then preform ocr on all the document first, then assign the texts using their coordinates to their respective cells, I found that this method works much better then applying ocr to each detected contour cell, I also added some code to fix broken lines in the table

In [ ]:
#set the start page and end page number
first_page=0
#last page +1
last_page=40
for counter in range(first_page,last_page):

  image_file=os.path.join(output_path,"page"+str(counter)+".jpg")
  image = cv2.imread(image_file)
  layout = model.detect(image)
  l=layout.to_dataframe()
  x_1=int(l["x_1"])
  x_2=int(l["x_2"])
  y_1=int(l["y_1"])
  y_2=int(l["y_2"])
  im =image[y_1:y_2, x_1:x_2]
  file_path=os.path.join(output_file,"page"+str(counter))
  os.mkdir(file_path)
  cv2.imwrite(file_path+"/original_page"+str(counter)+".jpg",image)
  cv2.imwrite(file_path+"/cropped_page"+str(counter)+".jpg",im)


  img=cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)

  #thresholding the image to a binary image
  blur = cv2.GaussianBlur(img,(5,5),0)
  thresh,img_bin = cv2.threshold(blur,128,255,cv2.THRESH_BINARY | cv2.THRESH_OTSU)

  #inverting the image 
  img_bin = 255-img_bin
  #Use horizontal kernel to detect and save the horizontal lines in a jpg
  hor_kernel=np.ones((1,120), np.uint8)
  image_2 = cv2.erode(img_bin, hor_kernel, iterations=3)
  horizontal_lines = cv2.dilate(image_2, hor_kernel, iterations=3)
  kernal_h = np.ones((1,50), np.uint8)
  closing_h= cv2.dilate(horizontal_lines, kernal_h, iterations=80)
  kernal_h = np.ones((2,4), np.uint8)
  eroded_h=cv2.erode(closing_h, kernal_h, iterations=1)
  ver_kernel=np.ones((70,1), np.uint8)
  #Use vertical kernel to detect and save the vertical lines in a jpg
  image_1 = cv2.erode(img_bin, ver_kernel, iterations=3)
  image2 = cv2.dilate(image_1, ver_kernel, iterations=20)
  vertical_lines = cv2.erode(image2, np.ones((2,2), np.uint8), iterations=3)
  # Combine horizontal and vertical lines in a new third image, with both having same weight.
  img_vh = cv2.addWeighted(vertical_lines, 0.5, eroded_h, 0.5, 0.0)
  #Eroding and thesholding the image
  kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))
  img_vh = cv2.erode(~img_vh, kernel, iterations=2)
  thresh, img_vh = cv2.threshold(img_vh,128,255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
  bitxor = cv2.bitwise_xor(img,img_vh)
  bitnot = cv2.bitwise_not(bitxor)
  img= im
  # Detect contours for following box detection
  contours, hierarchy = cv2.findContours(img_vh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

  def sort_contours(cnts, method="left-to-right"):
      # initialize the reverse flag and sort index
      reverse = False
      i = 0
      # handle if we need to sort in reverse
      if method == "right-to-left" or method == "bottom-to-top":
          reverse = True
      # handle if we are sorting against the y-coordinate rather than
      # the x-coordinate of the bounding box
      if method == "top-to-bottom" or method == "bottom-to-top":
          i = 1
      # construct the list of bounding boxes and sort them from top to
      # bottom
      boundingBoxes = [cv2.boundingRect(c) for c in cnts]
      (cnts, boundingBoxes) = zip(*sorted(zip(cnts, boundingBoxes),
      key=lambda b:b[1][i], reverse=reverse))
      # return the list of sorted contours and bounding boxes
      return (cnts, boundingBoxes)

  # Sort all the contours by top to bottom.
  contours, boundingBoxes = sort_contours(contours, method="top-to-bottom")


  #Creating a list of heights for all detected boxes
  heights = [boundingBoxes[i][3] for i in range(len(boundingBoxes))]

  #Get mean of heights
  mean = np.mean(heights)

  #Create list box to store all boxes in  
  box = []
  # Get position (x,y), width and height for every contour and show the contour on image
  for c in contours:
      x, y, w, h = cv2.boundingRect(c)
      # if (w<1000 and h<1000) and (x>1000 and y >1000):
      # image = 
      cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)
      box.append([x,y,w,h])
  #Creating two lists to define row and column in which cell is located
  row=[]
  column=[]
  j=0

  #Sorting the boxes to their respective row and column
  for i in range(len(box)):    
          
      if(i==0):
          column.append(box[i])
          previous=box[i]    
      
      else:
          if(box[i][1]<=previous[1]+mean/2):
              column.append(box[i])
              previous=box[i]            
              
              if(i==len(box)-1):
                  row.append(column)        
              
          else:
              row.append(column)
              column=[]
              previous = box[i]
              column.append(box[i])

  #calculating maximum number of cells
  countcol = 0
  for i in range(len(row)):
      countcol = len(row[i])
      if countcol > countcol:
          countcol = countcol

  #Retrieving the center of each column
  center = [int(row[i][j][0]+row[i][j][2]/2) for j in range(len(row[i])) if row[0]]

  center=np.array(center)
  center.sort()
  res = ocr_agent.detect(im, return_response=True)
  texts  = ocr_agent.gather_text_annotations(res) 
      # collect all the texts without coordinates
  layout = ocr_agent.gather_full_text_annotation(res, agg_level=lp.GCVFeatureType.WORD)
      # collect all the layout elements of the `WORD` level

  #Regarding the distance to the columns center, the boxes are arranged in respective order

  finalboxes = []
  for i in range(len(row)):
      lis=[]
      for k in range(countcol):
          lis.append([])
      for j in range(len(row[i])):
          diff = abs(center-(row[i][j][0]+row[i][j][2]/4))
          minimum = min(diff)
          indexing = list(diff).index(minimum)
          lis[indexing].append(row[i][j])
      finalboxes.append(lis)


  #from every single image-based cell/box the strings are extracted via pytesseract and stored in a list
  outer=[]
  for i in range(len(finalboxes)):
      for j in range(len(finalboxes[i])):
          inner=''
          if(len(finalboxes[i][j])==0):
              outer.append(' ')
          else:
              for k in range(len(finalboxes[i][j])):
                #get coorinates of the cell
                  y,x,w,h = finalboxes[i][j][k][0],finalboxes[i][j][k][1], finalboxes[i][j][k][2],finalboxes[i][j][k][3]
                  #get the text in those coordinates , here 
                  s=layout.filter_by(lp.Rectangle(y,x,y+w,x+h)).get_texts()
                  out = " ".join(ss for ss in s)
                  inner = inner +" "+ out
              outer.append(inner)

  #Creating a dataframe of the generated OCR list
  arr = np.array(outer)
  dataframe = pd.DataFrame(arr.reshape(len(row), countcol))
  # print(dataframe)
  data = dataframe.style.set_properties(align="left")
  #Converting it in a excel-file
  excel_path=os.path.join(file_path,"output_cropped"+str(counter)+".xlsx")
  data.to_excel(excel_path)